# Part2: RAG (Retrieval-Augmented Generation)
- 모델의 학습 데이터에 포함되지 않은 데이터를 사용 (환각 방지)
- **외부 데이터**를 검색(retrieval)한 후, 생성(generation) 단계에서 LLM에 전달
- 모델은 주어진 컨텍스트나 질문에 더 적합하고 풍부한 정보를 반영한 답변을 생성
- 논문: https://arxiv.org/abs/2005.11401

## 0. 환경 구성

### 1) 라이브러리 설치

In [1]:
!pip install -q langchain langchain_huggingface langchain_google_genai transformers sentence-transformers accelerate langchain_community pypdf faiss-cpu rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56

In [2]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

### Web based loader

In [3]:
loader = WebBaseLoader(
    web_paths=("https://www.bbc.com/news/business-68092814",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "main",
            attrs={"id": ["main-content"]},
        )
    ),
)
docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs[0].page_content[:500]

문서의 수: 1


'Could AI \'trading bots\' transform the world of investing?1 February 2024ShareSaveJonty BloomBusiness reporterShareSaveGetty ImagesIt is hard for both humans and computers to predict stock market movementsSearch for "AI investing" online, and you\'ll be flooded with endless offers to let artificial intelligence manage your money.I recently spent half an hour finding out what so-called AI "trading bots" could apparently do with my investments.Many prominently suggest that they can give me lucrative'

### PDF loader

In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/SPRI_AI_Brief_2023년12월호_F.pdf")

docs = loader.load()
print(f"문서의 수: {len(docs)}")

print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")

문서의 수: 23

[페이지내용]
SPRi AI Brief |  2023-12월호
8
코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개n코히어와 12개 기관이  광범위한 데이터셋에 대한 감사를 통해 원본 데이터 출처, 재라이선스 상태, 작성자 등 다양한 정보를 제공하는 ‘데이터 출처 탐색기’ 플랫폼을 출시n대화형 플랫폼을 통해 개발자는 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며 데이터셋의 구성과 계보도 추적 가능
KEY Contents
£데이터 출처 탐색기, 광범위한 데이터셋 정보 제공을 통해 데이터 투명성 향상nAI 기업 코히어(Cohere)가 매사추세츠 공과⼤(MIT), 하버드⼤ 로스쿨, 카네기멜론⼤ 등 12개 기관과 함께 2023년 10월 25일 ‘데이터 출처 탐색기(Data Provenance Explorer)’ 플랫폼을 공개∙AI 모델 훈련에 사용되는 데이터셋의 불분명한 출처로 인해 데이터 투명성이 확보되지 않아 다양한 법적·윤리적 문제가 발생∙이에 연구진은 가장 널리 사용되는 2,000여

[metadata]
{'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'author': 'dj', 'moddate': '2023-12-08T13:28:38+09:00', 'pdfversion': '1.4', 'source': '/content/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'page': 10, 'page_label': '11'}



### Text Splitter

In [5]:
from langchain.text_splitter import CharacterTextSplitter
loader = WebBaseLoader(
    web_paths=("https://www.bbc.com/news/business-68092814",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "main",
            attrs={"id": ["main-content"]},
        )
    ),
)

text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator=" ")

split_docs = loader.load_and_split(text_splitter=text_splitter)
print(f"문서의 수: {len(split_docs)}")
for i in range(5):
    print(split_docs[i].page_content[:500])

문서의 수: 152
Could AI 'trading bots' transform the world of
investing?1 February 2024ShareSaveJonty
BloomBusiness reporterShareSaveGetty ImagesIt is
hard for both humans and computers to predict
stock market movementsSearch for "AI investing"


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("/content/SPRI_AI_Brief_2023년12월호_F.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_docs = loader.load_and_split(text_splitter=text_splitter)

print(len(split_docs))
print(split_docs)

60
[Document(metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'author': 'dj', 'moddate': '2023-12-08T13:28:38+09:00', 'pdfversion': '1.4', 'source': '/content/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'page': 0, 'page_label': '1'}, page_content='2023년 12월호'), Document(metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'author': 'dj', 'moddate': '2023-12-08T13:28:38+09:00', 'pdfversion': '1.4', 'source': '/content/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'page': 1, 'page_label': '2'}, page_content='2023년 12월호'), Document(metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'author': 'dj', 'moddate': '2023-12-08T13:28:38+09:00', 'pdfversion': '1.4', 'source': '/content/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'page': 1, 

### Embedding

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(
    documents=split_docs, embedding=HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
)

query = "최근 인공지능의 동향이 뭐야?"

retriever = vectorstore.as_retriever(search_type="similarity")
search_result = retriever.get_relevant_documents(query)
print(search_result[0]['page_content'])

/tmp/ipython-input-7-3836871696.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  documents=split_docs, embedding=HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public 

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[Document(id='7a23cf8c-04f2-42f3-8e2f-a5b07923bb66', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'author': 'dj', 'moddate': '2023-12-08T13:28:38+09:00', 'pdfversion': '1.4', 'source': '/content/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'page': 2, 'page_label': '3'}, page_content='Ⅰ. 인공지능 산업 동향 브리프'), Document(id='9837d875-3dbd-43bf-88d7-248c7521ec77', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'author': 'dj', 'moddate': '2023-12-08T13:28:38+09:00', 'pdfversion': '1.4', 'source': '/content/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'page': 15, 'page_label': '16'}, page_content='1. 정책/법제  2. 기업/산업 3. 기술/연구  4. 인력/교육빌 게이츠, AI 에이전트로 인한 컴퓨터 사용의 패러다임 변화 전망n빌 게이츠가 5년 내 일상언어로 모든 작업을 처리할 수 있는 AI 에이전트가 보급되며 컴퓨터를 사용하는 방식이 완전히 바뀔 것으로 예상n에이전트의 보급은 컴퓨터 분야를 넘어 산업 전 영역에 영향을 미칠 전망으로 특히 의료와 교육, 생산성, 엔터테인먼트·쇼핑 영역에

/tmp/ipython-input-7-3836871696.py:12: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search_result = retriever.get_relevant_documents(query)


### Full step

In [8]:
from langchain.document_loaders import PyPDFLoader

file_path = "/content/SPRI_AI_Brief_2023년12월호_F.pdf"
loader = PyPDFLoader(file_path=file_path)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

split_docs = loader.load_and_split(text_splitter=text_splitter)

vectorstore = FAISS.from_documents(
    documents=split_docs, embedding=HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
)

k = 3

from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k = k

faiss_vectorstore = FAISS.from_documents(split_docs, HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask"))
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": k})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

prompt = hub.pull("rlm/rag-prompt")

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline

model_name = "Qwen/Qwen1.5-1.8B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model_hf = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
pipe = pipeline("text-generation", model=model_hf, tokenizer=tokenizer, max_new_tokens=512)
model = HuggingFacePipeline(pipeline=pipe)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-8-1095280627.py:53: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  model = HuggingFacePipeline(pipeline=pipe)


### Inference without RAG

In [9]:
question = "삼성 가우스에 대해 설명해주세요"
ans = pipe('<|im_start|>'+question)[0]['generated_text']
print(ans)

<|im_start|>삼성 가우스에 대해 설명해주세요慢慢的不断的深深的恐惧感的句子

1. 나는 두려움을 느끼며, 그것이 내 삶에서 가장 큰 문제를 겪고 있습니다.
2. 저는 그동안 많은 시간과 노력이 필요하며, 이 문제가 제대로 해결될 수 없습니다.
3. 나는 이러한 두려움에 대한 불편함을 느낄 수 있으며, 이를 통해 더 나은 세상을 만들어갈 수 있는 방법을 찾기 위해 노력하고 있습니다.
4. 이것은 내가 살던 모든 일의 두려움과 분노를 겪고 있습니다.
5. 나는 이렇게 두려움을 느끼는 것이 어려운 일이 많아서, 이를 극복하기 위한 계획을 세우고 있습니다.
6. 이러한 두려움에 대한 무단한 악력이 높았으며, 이를 극복하기 위해서는 많은 노력이 필요합니다.
7. 나는 이러한 두려움에 대한 불편함을 느끼며, 이를 통해 더 나은 세상을 만들 수 있는 방법을 찾습니다.
8. 이러한 두려움에 대한 무단한 악력을 가지고 있는 것은 매우 심각한 문제이며, 이를 극복하기 위해서는 많은 노력이 필요합니다.
9. 나는 이러한 두려움에 대한 불편함을 느끼며, 이를 통해 더 나은 세상을 만들 수 있는 방법을 찾습니다.
10. 이러한 두려움에 대한 불편함이 있어, 나는 더욱 심각한 문제로 인해, 이를 극복하기 위한 적절한 대처 방식을 찾아야 합니다.

번역결과
1. 나는 두려움을 느끼며, 그것은 내 삶에서 가장 큰 문제입니다.
2. 나는 이러한 두려움에 대한 불편함을 느끼며, 이를 통해 더 나은 세상을 만들 수 있는 방법을 찾습니다.
3. 이것은 내가 살던 모든 일의 두려움과 분노를 겪고 있습니다.
4. 이것은 이렇게 두려움을 느끼는 것이 어려운 일이 많아서, 이를 극복하기 위한 계획을 세우고 있습니다.
5. 이러한 두려움에 대한 무단한 악력을 가지고 있는 것은 매우 심각한 문제이고, 이를 극복하기 위해서는 많은 노력이 필요합니다.
6. 이러한 두려움에 대한 불편함을 느끼며, 이를 통해 더 나은 세상을 만들 수 있는 방법을 찾습니다.



### Inference with RAG

In [10]:
response = rag_chain.invoke(question)
print(f"PDF Path: {file_path}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

PDF Path: /content/SPRI_AI_Brief_2023년12월호_F.pdf
문서의 수: 23
[HUMAN]
삼성 가우스에 대해 설명해주세요

[AI]
Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: 삼성 가우스에 대해 설명해주세요 
Context: SPRi AI Brief |  2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 AI 모델 ‘삼성 가우스’를 공개n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents

KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원n삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 ‘삼성 가우스’를 최초 공개∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 최적화된 크기의 모델 선택이 가능∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 단계적으로 탑재할 계획n삼성 가우스는 △텍스